In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler

In [2]:
df = pd.read_csv('capital_bike_data/train.csv')

In [3]:
def feature_engin(df):
    df['date'] = pd.to_datetime(df['datetime'])
    df.set_index(['date'], inplace=True)
    df['year'] = df.index.year
    df['month'] = df.index.month
    df['day'] = df.index.day
    df['hour'] = df.index.hour
    
    season_dummys = pd.get_dummies(df['season'])
    weather_dummys = pd.get_dummies(df['weather'])
    df = pd.merge(left=df, right=season_dummys, how='inner', on=df.index).set_index('key_0')
    df = pd.merge(left=df, right=weather_dummys, how='inner', on=df.index).set_index('key_0')
    # drop one of new season and weather columns also!! (see alternative version in function 3)
    df.drop(['datetime', 'season', 'weather'], axis=1, inplace=True)
    df.reset_index(inplace=True, drop=True)

In [4]:
def feature_engin3(df):
    df['date'] = pd.to_datetime(df['datetime'])
    df.set_index(['date'], inplace=True)
#    df['year'] = df.index.year
    df['month'] = df.index.month
#    df['day'] = df.index.day
    df['hour'] = df.index.hour
    
    weather_columns = ['weather'] #the original column name from dataframe
    df = pd.get_dummies(df, columns=weather_columns, drop_first=True)
    season_columns = ['season']
    df = pd.get_dummies(df, columns=season_columns, drop_first=True)
    
    df.drop(['datetime'], axis=1, inplace=True)
    df.reset_index(inplace=True, drop=True)

## STEP 2. use Regression model

DONE:
- get a first impression & check out data
- find corrolations (heatmap & plotting)
- feature engineer

create your own features by binning data


## STEP 2.1. Split into test & train data

In [5]:
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [6]:
feature_engin3(df)
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count,month,hour
date,,,,,,,,,,,,,,
2011-01-01 00:00:00,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16,1,0
2011-01-01 01:00:00,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40,1,1
2011-01-01 02:00:00,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32,1,2
2011-01-01 03:00:00,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13,1,3
2011-01-01 04:00:00,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1,1,4


In [7]:
weather_columns = ['weather'] #the original column name from dataframe
df = pd.get_dummies(df, columns=weather_columns, drop_first=True)

In [8]:
df.head()

,datetime,season,holiday,workingday,temp,atemp,humidity,windspeed,casual,registered,count,month,hour,weather_2,weather_3,weather_4
date,,,,,,,,,,,,,,,,
2011-01-01 00:00:00,2011-01-01 00:00:00,1,0,0,9.84,14.395,81,0.0,3,13,16,1,0,0,0,0
2011-01-01 01:00:00,2011-01-01 01:00:00,1,0,0,9.02,13.635,80,0.0,8,32,40,1,1,0,0,0
2011-01-01 02:00:00,2011-01-01 02:00:00,1,0,0,9.02,13.635,80,0.0,5,27,32,1,2,0,0,0
2011-01-01 03:00:00,2011-01-01 03:00:00,1,0,0,9.84,14.395,75,0.0,3,10,13,1,3,0,0,0
2011-01-01 04:00:00,2011-01-01 04:00:00,1,0,0,9.84,14.395,75,0.0,0,1,1,1,4,0,0,0


In [ ]:
y = df['count'] #vector

In [ ]:
df = df.drop(['count', 'casual', 'registered'], axis=1)
X_features = df.columns
X = df[X_features] #matrix

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_log_error

from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
m = LinearRegression()
m_forest = RandomForestRegressor(max_depth=2, random_state=0, n_estimators=100)

In [ ]:
m.fit(X_train, y_train)
m_forest.fit(X_train, y_train)

## STEP 2.2 Prediction

In [ ]:
ypred = m.predict(X_test)
ypred_forest = m_forest.predict(X_test)

In [ ]:
ypred

In [ ]:
ypred_forest

In [ ]:
same = ypred == ypred_forest
same.sum()

In [ ]:
len(X_test)

## STEP 2.3 Score & Cross-Validation

In [ ]:
# for random forest model

In [ ]:
cvs = cross_val_score(m_forest, X_train, y_train, cv=10)
cvs.mean()

In [ ]:
cvs.std()

In [ ]:
m_forest.score(X_train, y_train)

In [ ]:
# for linear regression model

In [ ]:
# remember this scores does not work well with Regression Models so we use RMSLE

In [ ]:
# Squareroot does not work with negativ numbers, so ...

In [ ]:
ypred.min(), ypred.max(), y_test.min(), y_test.max(), 

In [ ]:
ypred[ypred <= 0] = 10

In [ ]:
ypred.min(), ypred.max()

In [ ]:
def rmsle(p,a):
    return np.sqrt(mean_squared_log_error([p],[a]))

In [ ]:
rmsle(y_test, ypred)

## STEP 3 Prediction w. test data

In [ ]:
df_t = pd.read_csv('capital_bike_data/test.csv')

In [ ]:
df_t.head()

In [ ]:
#feature_engin(df_t)
df_t.drop('temp', axis=1)
df_t.head()

In [ ]:
ypred_t_forest = m_forest.predict(df_t)

In [ ]:
ypred_t = m.predict(df_t)

In [ ]:
ypred_t[ypred_t<=0]=15
ypred_t_forest[ypred_t_forest<=0]=15

In [ ]:
ypred_t.min(), ypred_t_forest.min()

## STEP 4: submission & online verification 

In [ ]:
sub_df = pd.read_csv('capital_bike_data/sampleSubmission.csv')
df_t = pd.read_csv('capital_bike_data/test.csv')

In [ ]:
len(sub_df), len(df_t)

In [ ]:
ypred_t = np.round(ypred_t, decimals=0)
ypred_t_forest = np.round(ypred_t_forest, decimals=0)

In [ ]:
# for i in ypred_t np.int(i)

In [ ]:
sub_df['count'] = ypred_t

In [ ]:
#sub_df['count'][sub_df['count']==0.0 = 5  

In [ ]:
sub_df['count'] = sub_df['count'].astype(int)
sub_df.head()

In [ ]:
sub_df.reset_index(drop=True, inplace=True)

In [ ]:
sub_df.head()

In [ ]:
sub_df.to_csv('kaggel_submission_lin.csv', index=False)

In [ ]:
sub_df['count'] = ypred_t_forest
sub_df['count'] = sub_df['count'].astype(int)

In [ ]:
sub_df.to_csv('kaggel_submission_forest.csv', index=False)

In [ ]:
check = pd.read_csv('kaggel_submission_forest.csv')
check.head(30)

In [ ]:
check2 = pd.read_csv('kaggel_submission_lin.csv')
check2.head(30)